# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
!jupyter nbextension enable --py gmaps

In [47]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [48]:
#Load CSV file saved in Output Data
cities_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
cities_df.head()


,City ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,souillac,-20.5167,57.5167,78.80,83,40,8.05,MU,1612054156
1,1,shubarkuduk,49.1447,56.4872,15.42,94,0,9.37,KZ,1612054156
2,2,bukavu,-2.5083,28.8608,60.24,83,34,0.89,CD,1612054157
3,3,tuktoyaktuk,69.4541,-133.0374,-34.60,67,40,3.44,CA,1612054090
4,4,westport,41.1415,-73.3579,28.00,39,1,4.61,US,1612054157


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [49]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lon"]]
locations.head()
rating = cities_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [50]:
vacation_df = cities_df.loc[(cities_df["Max Temp"] < 85) &
                           (cities_df["Max Temp"] > 69) &
                           (cities_df["Wind Speed"] < 8) &
                           (cities_df["Humidity"] < 48)]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [74]:
#Create column hotel name and store in DF called hotel_df
vacation_df["Hotel Name"] = ""
hotel_df = vacation_df
hotel_df

#Set parameters
params = {"radius": 5000,
         "types": "lodging",
         "key": g_key}

#Hit Google Places API for each city coordinates
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lon = row["Lon"]
    
    params["location"] = f"{lat},{lon}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params = params)
    address = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = address["results"][0]['name']
    except (KeyError, IndexError):
        print('Missing field/result... skipping')


<ipython-input-74-8f5c3e4ca523>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacation_df["Hotel Name"] = ""


Missing field/result... skipping


/Users/chelseaball/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping


In [75]:
hotel_df

,City ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
35,35,siguiri,11.6667,-9.5000,69.80,21,0,2.71,GN,1612054118,
50,50,mataura,-46.1927,168.8643,71.98,37,96,3.56,NZ,1612054167,Ellie's Villa
150,150,del rio,29.3627,-100.8968,78.80,24,1,6.91,US,1612053976,Holiday Inn Express & Suites del Rio
173,173,new norfolk,-42.7826,147.0587,75.00,26,4,4.00,AU,1612054054,The Shingles Riverside Cottages
500,500,zapote,21.0667,-102.8000,71.65,22,3,6.85,MX,1612054271,LOS YUGOS
525,525,ouadda,8.0777,22.4007,70.34,22,42,7.34,CF,1612054276,


In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]
locations

,Lat,Lon
35,11.6667,-9.5000
50,-46.1927,168.8643
150,29.3627,-100.8968
173,-42.7826,147.0587
500,21.0667,-102.8000
525,8.0777,22.4007


In [79]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))